<a href="https://colab.research.google.com/github/wa1575/titanic_learning/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive/')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
drive_path = "/gdrive/MyDrive/"
train = pd.read_csv(drive_path + "titanic/train.csv")
test = pd.read_csv(drive_path + "titanic/test.csv")
submission = pd.read_csv(drive_path + "titanic/sample_submission.csv")
print(train.shape, test.shape, submission.shape)

In [ ]:
train.head(3)

In [ ]:
test.head(2)

In [ ]:
submission.head()

In [ ]:
train.info()

In [ ]:
train.describe(include='all')

In [ ]:
import missingno as msno
msno.bar(train, figsize=(10,5), color=(0.7, 0.2, 0.2))
plt.show()

In [ ]:
msno.matrix(test, figsize=(10,5), color=(0.7, 0.2, 0.2))
plt.show()

In [ ]:
#숫자형 변수 간의 상관 관계를 계산하여 히트맵 그리기
plt.figure(figsize=(8,8))
sns.set(font_scale=0.8)
sns.heatmap(train.corr(), annot = True, cbar=True);
plt.show()

In [ ]:
#타이타닉 전체 데이터셋 준비
train['TrainSplit'] = 'Train'
test['TrainSplit'] = 'Test'
data = pd.concat([train,test], axis=0)
print(data.shape)

In [ ]:
#숫자형 피처 추출
data_num = data.loc[:,['Pclass', 'Age','SibSp', 'Parch', 'Fare','Survived']]

#결측값 대체
data_num['Age'] = data_num['Age'].fillna(data_num['Age'].mean())
data_num['Fare'] = data_num['Fare'].fillna(data_num['Fare'].mode()[0])

#학습용 데이터와 예측 대상인 테스트 데이터 구분
selected_features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
x_train = data_num.loc[data['TrainSplit']=='Train', selected_features]
y_train = data_num.loc[data['TrainSplit']=='Train', 'Survived']

x_test = data_num.loc[data['TrainSplit']=='Test', selected_features]

print("Train 데이터셋 크기:", x_train.shape, y_train.shape)
print("Test 데이터셋 크기:", x_test.shape)

In [ ]:
#훈련 - 검증 데이터 분할
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, test_size = 0.2,
                                            shuffle=True, random_state=20)

#로지스틱 회귀 모델
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()
lr_model.fit(x_tr, y_tr)
y_val_pred = lr_model.predict(x_val)

#Confusion Matrix
from sklearn.metrics import confusion_matrix
sns.heatmap(confusion_matrix(y_val, y_val_pred), annot = True, cbar = False,
            square=True)
plt.show()

In [ ]:
#평가지표
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score
print("Accuracy:%.4f" % accuracy_score(y_val, y_val_pred))
print("Precision:%.4f" % precision_score(y_val, y_val_pred))
print("Recall:%.4f" % recall_score(y_val, y_val_pred))
print("F1:%.4f" % f1_score(y_val, y_val_pred))
print("AUC:%.4f" % roc_auc_score(y_val, y_val_pred))

In [ ]:
#test 데이터에 대한 예측값 정리 
y_test_pred = lr_model.predict(x_test)

#제출양식에 맞게 정리
submission['Survived'] = y_test_pred.astype(int)

#제출 파일 저장
submission_filepath = drive_path + 'baseline_num_lr_submission_001.csv'
submission.to_csv(submission_filepath, index=False)
submission.head(5)